# 5. 데코레이터를 활용한 코드 개선
+ 데코레이터 패턴과 혼동하지 말 것

## 함수 데코레이터

In [2]:
""" 도메인의 특정 예외에 대해서 특정 횟수만큼 재시도하는 데코레이터 """

from functools import wraps
from random import randint
import logging

class ControlledException(Exception):
    """ 일반적인 예외 가정 """

def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for i in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised
    return wrapped


@retry
def run_operation(a:int, b:int):
    if randint(a, b) % 2 == 0:
        print("Completed a function")
    else:
        raise ControlledException()

if __name__ == "__main__":
    logger = logging.getLogger('example')
    logger.setLevel(logging.DEBUG)
    run_operation(0, 9)

Completed a function


## 클래스 데코레이터
+ 파라미터로 함수가 아닌 클래스를 받는다는 점만 차이점이 있음
+ 장점
    + 코드 재사용과 DRY 원칙의 모든 이점을 공유할 수 있음
    + 당장은 작고 간단한 클래스를 생성하고 나중에 데코레이터로 기능을 보강할 수 있음
    + 기존 로직을 쉽게 변경 가능하며, 메타클래스와 같은 방법보다 비교적 간단함

In [3]:
""" 확장에 불리한 구조: 이벤트 클래스와 직렬화 클래스가 1대 1로 매팅됨 """

class LoginEventSerializer:
    def __init__(self, event):
        self.event = event

    def serialize(self) -> dict:
        return {
            "username": self.event.username,
            "password": len(self.event.password) * "*",
            "ip": self.event.ip,
        }


class LoginEvent:
    SERIALIZER = LoginEventSerializer

    def __init__(self, username, password, ip):
        self.username = username
        self.password = password
        self.ip = ip

    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()


if __name__ == "__main__":
    event = LoginEvent("shyeon", "0524", "127.0.0.1")
    print(event.serialize())

{'username': 'shyeon', 'password': '****', 'ip': '127.0.0.1'}


In [4]:
""" 데코레이션을 활용한 구조 개선 """

from datetime import datetime

def hide_field(field) -> str:
    return len(field) * "*"

def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")

def show_original(event_field):
    return event_field


class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields

    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation in self.serialization_fields.items()
        }


class Serialization:
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)

    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)
        event_class.serialize = serialize_method
        return event_class


@Serialization(
    username=show_original,
    password=hide_field,
    ip=show_original,
    timestamp=format_time)
class LoginEvent:
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp

if __name__ == "__main__":
    event = LoginEvent("shyeon", "0524", "127.0.0.1", datetime(2020, 2, 14, 9, 00, 00))
    print(event.serialize())

{'username': 'shyeon', 'password': '****', 'ip': '127.0.0.1', 'timestamp': '2020-02-14 09:00'}


In [6]:
""" dataclass 데코레이터: init 함수의 단순코드 작성 간편화 """

from dataclasses import dataclass
from datetime import datetime

@Serialization(
    username=show_original,
    password=hide_field,
    ip=show_original,
    timestamp=format_time)
@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: datetime

if __name__ == "__main__":
    event = LoginEvent("shyeon", "0524", "127.0.0.1", datetime(2020, 2, 14, 9, 00, 00))
    print(event.serialize())

{'username': 'shyeon', 'password': '****', 'ip': '127.0.0.1', 'timestamp': '2020-02-14 09:00'}


## 데코레이터에 인자 전달
+ 간접 참조를 통해 새로운 레벨의 중첩 함수를 만들어 데코레이터의 모든 것을 한 단계 더 깊게 만듬
+ 데코레이터를 위한 클래스를 생성(가독성이 더 좋음)

### 중첩함수를 사용하는 방법

In [14]:
RETRIES_LIMIT = 3

def with_retry(retries_limit=RETRIES_LIMIT, allowed_exceptions=None): # 1.인수를 전달
    allowed_exceptions = allowed_exceptions or (ControlledException,)

    def retry(operation): # 2.데코레이터가 될 함수
        @wraps(operation)

        def wrapped(*arg, **kwargs): # 3.결과를 반환
            last_raised = None
            for _ in range(retries_limit):
                try:
                    return operation(*arg, **kwargs)
                except allowed_exceptions as e:
                    logger.info("retrying %s due to %s", operation, e)
                    last_raised = e
            raise last_raised


@with_retry(retries_limit=3)
def run_operation(a:int, b:int):
    if randint(a, b) % 2 == 0:
        print("Completed a function")
    else:
        raise ControlledException()


if __name__ == "__main__":
    logger = logging.getLogger('example')
    logger.setLevel(logging.DEBUG)
    run_operation(0, 9)

TypeError: 'NoneType' object is not callable

### 데코레이터 객체

In [15]:
RETRIES_LIMIT = 3


class WithRetry:
    def __init__(self, retries_limit=RETRIES_LIMIT, allowed_exceptions=None):
        self.retries_limit = retries_limit
        self.allowed_exceptions = allowed_exceptions or (ControlledException,)

    def __call__(self, operation):
        @wraps(operation)
        def wrapped(*arg, **kwargs): # 3.결과를 반환
            last_raised = None
            for _ in range(self.retries_limit):
                try:
                    return operation(*arg, **kwargs)
                except self.allowed_exceptions as e:
                    logger.info("retrying %s due to %s", operation, e)
                    last_raised = e
            raise last_raised


@WithRetry(retries_limit=RETRIES_LIMIT)
def run_operation(a:int, b:int):
    if randint(a, b) % 2 == 0:
        print("Completed a function")
    else:
        raise ControlledException()
        

if __name__ == "__main__":
    logger = logging.getLogger('example')
    logger.setLevel(logging.DEBUG)
    run_operation(0, 9)

TypeError: 'NoneType' object is not callable